In [1]:
#%
import numpy as np
import pandas as pd
import glob
import os

In [2]:
a = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_selected_mmd_lowercase_aus.txt',dtype='str')

In [ ]:
#%% GSWP PR
fol1='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/grid_to_catchments/out_csv_gswp_p/all_years'

for i in range(len(a)):
    b = a[i]
    f = f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_gswp_timeseries_selected_catchments/daily/{b}_gswp_p_1981_2010.csv'
    if os.path.isfile(f):
        continue
    else:
        l = glob.glob(fol1 + f"*/{b}*.csv") # all files from catchment b
        if len(l)==0:
            continue
        li=[]
        for filename in l:
            df = pd.read_csv(filename, index_col=1, header=0)
            df = df.drop(columns=['Unnamed: 0'])
            df.index = pd.to_datetime(df.index)
            li.append(df)

        frame = pd.concat(li, axis=0, ignore_index=False)
        y_start,y_end = frame.index[0].year, frame.index[-1].year
        frame.to_csv(f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_gswp_timeseries_selected_catchments/daily/{b}_gswp_p_{y_start}_{y_end}.csv')
        # print(i)


In [10]:
#%% GLEAM
fol1='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/grid_to_catchments/out_csv_gleam_ep/all_years'

for i in range(len(a)):
    b = a[i]
    f = f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/ep_gleam_timeseries_selected_catchments/daily/{b}_gleam_ep_*.csv'
    if os.path.isfile(f):
        continue
    else:
        l = glob.glob(fol1 + f"*/{b}*.csv") # all files from catchment b
        if len(l)==0:
            continue
        li=[]
        for filename in l:
            df = pd.read_csv(filename, index_col=1, header=0)
            df = df.drop(columns=['Unnamed: 0'])
            df.index = pd.to_datetime(df.index)
            li.append(df)

        frame = pd.concat(li, axis=0, ignore_index=False)
        y_start,y_end = frame.index[0].year, frame.index[-1].year
        frame.to_csv(f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/ep_gleam_timeseries_selected_catchments/daily/{b}_gleam_ep_{y_start}_{y_end}.csv')

In [17]:
# get yearly timeseries P
fol='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_gswp_timeseries_selected_catchments/daily'
for i in range(len(a)):
    b = a[i]
    l = glob.glob(fol + f"*/{b}*.csv") # all files from catchment b
    df = pd.read_csv(l[0], index_col=0)
    df.index = pd.to_datetime(df.index)
    df_y = df.groupby(pd.Grouper(freq='Y')).mean()
    y_start,y_end = df_y.index[0].year, df_y.index[-1].year
    df_y.to_csv(f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_gswp_timeseries_selected_catchments/yearly/{b}_gswp_p_{y_start}_{y_end}.csv')


In [11]:
# get yearly timeseries EP
fol='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/ep_gleam_timeseries_selected_catchments/daily'
for i in range(len(a)):
    b = a[i]
    l = glob.glob(fol + f"*/{b}*.csv") # all files from catchment b
    df = pd.read_csv(l[0], index_col=0)
    df.index = pd.to_datetime(df.index)
    df_y = df.groupby(pd.Grouper(freq='Y')).mean()
    y_start,y_end = df_y.index[0].year, df_y.index[-1].year
    df_y.to_csv(f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/ep_gleam_timeseries_selected_catchments/yearly/{b}_gleam_ep_{y_start}_{y_end}.csv')


In [12]:
# combine dfs Q P EP
a = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_selected_mmd_lowercase_aus.txt',dtype='str')
b = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_selected_mmd_aus.txt',dtype='str')
e = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_aus.txt',dtype='str')

fol='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/'
for i in range(len(a)):
    c = a[i]
    d = b[i]
    
    f = f'{fol}/p_q_ep_timeseries_selected_catchments/yearly/{c}.csv'
    if os.path.isfile(f):
        continue
    else:
        l = glob.glob(f'{fol}/p_gswp_timeseries_selected_catchments/yearly/{c}*.csv')
        df_p = pd.read_csv(l[0], index_col=0)
        df_p.index = pd.to_datetime(df_p.index)
        # print(df_p)

        l = glob.glob(f'{fol}/ep_gleam_timeseries_selected_catchments/yearly/{c}*.csv')
        df_ep = pd.read_csv(l[0], index_col=0)
        df_ep.index = pd.to_datetime(df_ep.index)
        # print(df_ep)

        l = glob.glob(f'{fol}/q_timeseries_selected_catchments/yearly/{d}*.csv')
        df_q = pd.read_csv(l[0], index_col=0)
        df_q.index = pd.to_datetime(df_q.index)
        # print(df_q)
        
        #if aus catchments:
        if d in e:
            df_q = df_q.dropna()
            df_q = df_q.rename(columns={'q_mmd':'mean_mmd'})

        # get longest timeseries
        p_s, p_e = df_p.index[0], df_p.index[-1]
        ep_s, ep_e = df_ep.index[0], df_ep.index[-1]
        q_s, q_e = df_q.index[0], df_q.index[-1]
        df_s, df_e = max(p_s, ep_s, q_s), min(p_e, ep_e, q_e)

        # make combined dataframe
        df = pd.DataFrame(index = pd.date_range(df_s, df_e, freq='Y'), columns=['q','p','ep'])
        df.p = df_p['precipitation_flux']
        df.ep = df_ep['Potential evaporation from GLEAM v3.5a']
        df.q = df_q['mean_mmd']
        df.to_csv(f'{fol}/p_q_ep_timeseries_selected_catchments/yearly/{c}.csv')


In [13]:
# long term means
fol='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/'
df_m = pd.DataFrame(index=a, columns=['q','p','ep','y_start','y_end'])

for i in range(len(a)):
    b = a[i]
    l = glob.glob(f'{fol}p_q_ep_timeseries_selected_catchments/yearly/{b}*.csv')
    df = pd.read_csv(l[0], index_col=0)
    df.index = pd.to_datetime(df.index)
    df_m.loc[b].q = df.q.mean()
    df_m.loc[b].ep = df.ep.mean()
    df_m.loc[b].p = df.p.mean()
    df_m.loc[b].y_start = df.index[0].year
    df_m.loc[b].y_end = df.index[-1].year
df_m.to_csv(f'{fol}/p_q_ep_timeseries_selected_catchments/mean_q_p_ep.csv')

In [14]:
# combine p and ep daily dataframes
folep='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/ep_gleam_timeseries_selected_catchments/daily'
folp='/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_gswp_timeseries_selected_catchments/daily'
a = np.genfromtxt('/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/catch_id_selected_mmd_lowercase_aus.txt',dtype='str')

for i in range(len(a)):
    b = a[i]
    
    l = glob.glob(folep + f"*/{b}*.csv") # all files from catchment b
    df_ep = pd.read_csv(l[0], index_col=0)
    df_ep.index = pd.to_datetime(df_ep.index)
    
    l = glob.glob(folp + f"*/{b}*.csv") # all files from catchment b
    df_p = pd.read_csv(l[0], index_col=0)
    df_p.index = pd.to_datetime(df_p.index)
    
    # get longest timeseries
    p_s, p_e = df_p.index[0], df_p.index[-1]
    ep_s, ep_e = df_ep.index[0], df_ep.index[-1]
    df_s, df_e = max(p_s, ep_s), min(p_e, ep_e)
    
    df = pd.DataFrame(index = pd.date_range(df_s, df_e, freq='d'), columns=['p','ep'])
    df.p = df_p['precipitation_flux']
    df.ep = df_ep['Potential evaporation from GLEAM v3.5a']
    
    df.to_csv(f'/home/vanoorschot/work/fransje/scripts/GLOBAL_SR/p_ep_timeseries_selected_catchments/daily/{b}.csv')
    
    

In [15]:
print(df)

                       p        ep
1981-01-01  0.000000e+00  4.967766
1981-01-02  0.000000e+00  4.300149
1981-01-03  0.000000e+00  4.303851
1981-01-04  0.000000e+00  4.779653
1981-01-05  4.307608e-04  4.477086
...                  ...       ...
2009-12-27  0.000000e+00  3.856536
2009-12-28  0.000000e+00  4.854429
2009-12-29  1.304845e-01  5.020740
2009-12-30  4.568194e-07  4.965876
2009-12-31  4.815277e-02  4.944347

[10592 rows x 2 columns]
